In [5]:
import pandas as pd
from bs4 import BeautifulSoup
import requests


from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
# -----------------------------------------------------------------------

pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

In [35]:
lista_pelis = [("Drama","Movie","The Drop",2006,6,"tt0340084"),("Thriller","Short","Geocatching",2015,11.0,"tt5142104")]


lista_imdb =[]

lista_tupla_actores=[]



driver = webdriver.Chrome()

driver.get("https://www.imdb.com/")

driver.maximize_window()


for tupla in lista_pelis:
    

    sleep(2)
    #Escribe el título de la pelicula en el buscador
    driver.find_element("css selector", "#suggestion-search").send_keys(tupla[-1], Keys.ENTER)

    sleep(2)
    # Puntución IMDB
    try:
      
        imdb_puntos = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div.sc-3a4309f8-0.fjtZsE.sc-dffc6c81-1.fJrHDo > div > div:nth-child(1) > a > span > div > div.sc-bde20123-0.gtEgaf > div.sc-bde20123-2.gYgHoj > span.sc-bde20123-1.iZlgcd").text
    except:
        imdb_puntos = "NO DATA"
    print(imdb_puntos)

    # Director IMDB
    # está dando error al appendear: 
    # TypeError: 'builtin_function_or_method' object is not subscriptable

    try:
        imdb_director = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > div > section > div > div.sc-9178d6fe-1.kFxVZc.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.gmonkL.title-cast.title-cast--movie.celwidget > ul > li:nth-child(1) > div > ul > li > a").text
        print(imdb_director)

    except:
        imdb_director = "NO DATA"

    # Guionistas IMDB
    guionistas = []
    for i in range(1,6):
        try: 
            imdb_guion = driver.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > div > section > div > div.sc-9178d6fe-1.kFxVZc.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.gmonkL.title-cast.title-cast--movie.celwidget > ul > li:nth-child(2) > div > ul > li:nth-child({i}) > a").text

            guionistas.append(imdb_guion)
        except:
            break
    print(guionistas)
    
        
    sleep(2)

    # Argumento IMDB
    # Es importante dejar abrir la pagina en maximize porque si no no nos lo coge el css selector.
    try:
        imdb_argumento = driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-4.ecgcFy > div.sc-e226b0e3-6.hfusNC > div.sc-e226b0e3-10.jcOwsU > section > p > span.sc-466bb6c-2.eVLpWt').text
        print(imdb_argumento)
    except:
        imdb_argumento = "NO DATA"    


    # Minutos IMDB
    try:
        imdb_minutos = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div.sc-dffc6c81-0.iwmAVw > ul > li:nth-child(3)").text

        lista_min = imdb_minutos.split()
    
        #print(lista_min)
        try: 
            #Ponemos un try...except porque nos hemos dado cuenta de que algunas películas tienen otra característica en la poscición de "hora,minutos"
            # Por ejemplo: 
            # __next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div.sc-dffc6c81-0.iwmAVw > ul > li:nth-child(4)  
            
            if len(lista_min) > 1:
                hora = int(lista_min[0].replace('h', ''))
                hora 
                min = int(lista_min[1].replace('min', ''))
                duracion = 60*hora + min
                print(duracion)
                            
            else: 
                duracion= int(lista_min[0].replace('min',''))
                duracion 
                print(duracion)

        except:
                duracion = "Formato min incorrecto"

    except:
        duracion = "NO DATA"            


    lista_actores = [tupla[-1]]
    for i in range(1, 11):
        sleep(2)
        # cogemos xpath porque el selector no nos cogía bien los nombres
        try:
            lista_actores.append(driver.find_element("xpath", f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[3]/div[2]/div[2]/div[{i}]/div[2]/a').text)
   
        except:
            break
        
    lista_tupla_actores.append(tuple(lista_actores))      
    lista_imdb.append((tupla[2],imdb_puntos,imdb_director,guionistas,imdb_argumento,duracion,lista_actores))

    print(lista_imdb)

    driver.back()

driver.close()

2,7
Kevin Lewis
['Kevin Lewis', 'Troy Scott']
A college student is hired to simply deliver a car to an LA parking garage. The job sounds too easy, so he checks the trunk and becomes a target in a secret war between forces of Heaven and Hell who are after the contents of the trunk.
88
[('The Drop', '2,7', 'Kevin Lewis', ['Kevin Lewis', 'Troy Scott'], 'A college student is hired to simply deliver a car to an LA parking garage. The job sounds too easy, so he checks the trunk and becomes a target in a secret war between forces of Heaven and Hell who are after the contents of the trunk.', 88, ['tt0340084', 'John Savage', 'Sean Young', 'Michael Bondies', 'Kat Ogden', 'Marc Vann', 'Kevan Michaels', 'Eric Livingstone', 'Greg Joelson', 'George Asher', 'Michael Todd'])]
NO DATA
Jennifer DiMarco
['Jennifer DiMarco']
[('The Drop', '2,7', 'Kevin Lewis', ['Kevin Lewis', 'Troy Scott'], 'A college student is hired to simply deliver a car to an LA parking garage. The job sounds too easy, so he checks t

In [ ]:
 # pruebas de actores y argumento
print(lista_actores)
print(lista_tupla_actores)

print(imdb_argumento)

In [ ]:
## ejemplos 
print(duracion)
print(dict_imdb)

In [36]:
# ACTORES dataframe:

df = pd.DataFrame(lista_tupla_actores)
df


,0,1,2,3,4,5,6,7,8,9,10
0,tt0340084,John Savage,Sean Young,Michael Bondies,Kat Ogden,Marc Vann,Kevan Michaels,Eric Livingstone,Greg Joelson,George Asher,Michael Todd
1,tt5142104,None,None,None,None,None,None,None,None,None,None


In [ ]:
## ROTTENTOMATOES
# Faltaría poner en googlechrome

diccionario = {'Genero': ["Drama"],
                    'Tipo' : ["Movie"],
                    'Nombre' : ["The Hunger Games"],
                    'Año estreno' : [2012],
                    'Mes estreno' : [12],
                    'Id' :['tt0313462']}

driver = webdriver.Chrome()

driver.get("https://www.rottentomatoes.com/")

driver.maximize_window()

#cookies
driver.find_element("css selector", '#onetrust-reject-all-handler').click()

#inputs película
#Escribe el título de la pelicula en el buscador
driver.find_element("css selector", "#header-main > search-algolia > search-algolia-controls > input").send_keys(f'"{diccionario["Nombre"][0]}"', Keys.ENTER)
sleep(4)
año = driver.find_element("xpath", f'//*[@id="search-results"]/search-page-result[2]/ul/search-page-media-row[1]//li/div[3]/span[1]').text

año

In [ ]:
# sleep(3)
# for i in range (1, 11):
#     sleep(3)
#     año = driver.find_element("xpath", f'//*[@id="search-results"]/search-page-result[2]/ul/search-page-media-row[{i}]//li/div[3]/span[1]').text
#     sleep(3)
#     if año == diccionario['Año estreno']:
#         rotten_puntos = driver.find_element("xpath", f'//*[@id="search-results"]/search-page-result[2]/ul/search-page-media-row[{i}]//li/div[1]/score-icon-critic-deprecated//div/span[2]').text
# li > div.info-wrap > span.year
# li > div.info-wrap > span.year
# li > div.info-wrap > span.year 

año
# //*[@id="search-results"]/search-page-result[2]/ul/search-page-media-row[1]//li/div[3]/span[1]
# //*[@id="search-results"]/search-page-result[2]/ul/search-page-media-row[2]//li/div[3]/span[1]
# //*[@id="search-results"]/search-page-result[2]/ul/search-page-media-row[3]//li/div[3]/span[1] 

# //*[@id="search-results"]/search-page-result[2]/ul/search-page-media-row[3]//li/div[1]/score-icon-critic-deprecated//div/span[2]
# //*[@id="search-results"]/search-page-result[2]/ul/search-page-media-row[4]//li/div[1]/score-icon-critic-deprecated//div/span[2]

In [ ]:
print(imdb_puntos)
print(imdb_director)
print(guionistas)